# Análisis de redes mediante matriz admitancia indefinida
<img src="./logo_UTN.svg" align="right" width="150" /> 

#### Por Mariano Llamedo Soria

### Resumen 
En este documento se presentan ...

* Funciones de dibujo de redes: [dibujar_Pi](), [dibujar_Tee]()
* Funciones de ... 
* Funciones para presentación de markdown y latex: [print_latex](), [print_subtitle](), [a_equal_b_latex_s]()


### Introducción

Documento en elaboración.



### T-puenteado de R constante

In [3]:
"""
Matriz Admitancia Indefinida (MAI)
----------------------------------
Ejemplos de cálculo simbólico mediante MAI de una red T puenteada de R constante.

Referencias:
------------
Cap. 9. Avendaño L. Sistemas electrónicos Analógicos: Un enfoque matricial.
"""

import sympy as sp

from pytc2.cuadripolos import calc_MAI_impedance_ij, calc_MAI_vtransf_ij_mn, calc_MAI_ztransf_ij_mn
from pytc2.general import print_latex


# T puenteado cargado: red de R constante
# explicación:
'''    
+ Numeramos los polos de 0 a n=3

            |------Ya-------|
            |               |
    0-------+--G----2---G---3
                    |       |
                   Yb       G
                    |       |
    1---------------+--------
    
'''    

Ya, Yb = sp.symbols('Ya Yb', complex=True)
G = sp.symbols('G', real=True, positive=True)

# Armo la MAI

#               Nodos: 0      1        2        3
Ymai = sp.Matrix([  
                    [ Ya+G,   0,      -G,     -Ya],
                    [ 0,     Yb+G,    -Yb,    -G],
                    [ -G,   -Yb,      2*G+Yb, -G],
                    [ -Ya,   -G,     -G,      2*G+Ya ]
                 ])

con_detalles = False
# con_detalles = True

# Calculo la Z en el puerto de entrada a partir de la MAI
Zmai = calc_MAI_impedance_ij(Ymai, 0, 1, verbose=con_detalles)

# Aplico la condición de R constante
print('si consideramos:')
print_latex( r'G^2 = Y_a . Y_b' )
print('entonces')
print_latex( r'Z_{{ {:d}{:d} }} = '.format(0,1) +  sp.latex(Zmai.subs(Ya*Yb, G**2)) )

print('Transferencia de tensión:')
Vmai = calc_MAI_vtransf_ij_mn(Ymai, 3, 1, 0, 1, verbose=con_detalles)
Vmai = sp.simplify(Vmai.subs(Ya*Yb, G**2))
Vmai_Ya = sp.simplify(Vmai.subs(Yb, G**2/Ya))
Vmai_Yb = sp.simplify(Vmai.subs(Ya, G**2/Yb))

print_latex( r'T^{{ {:d}{:d} }}_{{ {:d}{:d} }} = '.format(3, 1, 0, 1) +  sp.latex(Vmai_Ya) + ' = ' + sp.latex(Vmai_Yb) )

print('Transimpedancia:')
Zmai = calc_MAI_ztransf_ij_mn(Ymai, 3, 1, 0, 1, verbose=con_detalles)
Zmai = sp.simplify(Zmai.subs(Ya*Yb, G**2))
Zmai_Ya = sp.simplify(Zmai.subs(Yb, G**2/Ya))
Zmai_Yb = sp.simplify(Zmai.subs(Ya, G**2/Yb))
print_latex( r'Z^{{ {:d}{:d} }}_{{ {:d}{:d} }} = '.format(3, 1, 0, 1) + sp.latex(Zmai_Ya) + ' = ' + sp.latex(Zmai_Yb) )



si consideramos:


<IPython.core.display.Math object>

entonces


<IPython.core.display.Math object>

Transferencia de tensión:


<IPython.core.display.Math object>

Transimpedancia:


<IPython.core.display.Math object>

### Butterworth orden 3

In [ ]:

'''    
+ Numeramos los polos de 0 a n=3

    0-------+--Y1----2---Y3--3---
                     |           /
                    Y2           / R
                     |           /
    1----------------+-------1----
    
'''    

# definición de puertos. El primer nodo marca el sentido positivo de la tensión. V1 = V[input_port[0]] - V[input_port[1]]
input_port = [0, 1]
output_port = [3, 1]

Y1, Y2, Y3 = sp.symbols('Y1 Y2 Y3', complex=True)
G = sp.symbols('G', real=True, positive=True)

# Armo la MAI

#               Nodos: 0      1        2        3
Ymai = sp.Matrix([  
                    [ Y1,    0,      -Y1,      0],
                    [ 0,    Y2+G,    -Y2,     -G],
                    [ -Y1,  -Y2,    Y1+Y2+Y3, -Y3],
                    [ 0,    -G,      -Y3,      Y3+G ]
                 ])

s = sp.symbols('s ', complex=True)

# Butter de 3er orden simplemente cargado
Ymai = Ymai.subs(Y1, 1/s/sp.Rational('3/2'))
Ymai = Ymai.subs(Y3, 1/s/sp.Rational('1/2'))
Ymai = Ymai.subs(Y2, s*sp.Rational('4/3'))

# Butter de 3er orden doblemente cargado
# Ymai = Ymai.subs(Y1, 1/s/sp.Rational('1'))
# Ymai = Ymai.subs(Y3, 1/s/sp.Rational('1'))
# Ymai = Ymai.subs(Y2, s*sp.Rational('2'))
Ymai = Ymai.subs(G, sp.Rational('1'))

# con_detalles = False
con_detalles = True

# Calculo la Z en el puerto de entrada a partir de la MAI
Zmai = calc_MAI_impedance_ij(Ymai, input_port[0], input_port[1], verbose=con_detalles)

print_latex( r'Z_{{ {:d}{:d} }} = '.format(0,1) +  sp.latex(Zmai) )


print('Transferencia de tensión:')
Vmai = calc_MAI_vtransf_ij_mn(Ymai, output_port[0], output_port[1], input_port[0], input_port[1], verbose=con_detalles)
Vmai_sym = sp.simplify(Vmai.subs(Y3, Y1))

print_latex( r'T^{{ {:d}{:d} }}_{{ {:d}{:d} }} = '.format(output_port[0], output_port[1], input_port[0], input_port[1]) +  sp.latex(Vmai) )


